## 🙋‍♀️🙋‍♂️ Employee Termination Prediction

Given *employee data from HR*, let's try to predict which employees are most likely to be **terminated**.

We will use various classification models to make our predictions.

Data source: https://www.kaggle.com/datasets/rhuebner/human-resources-data-set

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
data = pd.read_csv('HRDataset_v14.csv')
data

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,Termd,PositionID,Position,State,Zip,DOB,Sex,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc,DateofHire,DateofTermination,TermReason,EmploymentStatus,Department,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506,0,19,Production Technician I,MA,1960,07/10/83,M,Single,US Citizen,No,White,7/5/2011,NaN,N/A-StillEmployed,Active,Production,Michael Albert,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437,1,27,Sr. DBA,MA,2148,05/05/75,M,Married,US Citizen,No,White,3/30/2015,6/16/2016,career change,Voluntarily Terminated,IT/IS,Simon Roup,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17
2,"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955,1,20,Production Technician II,MA,1810,09/19/88,F,Married,US Citizen,No,White,7/5/2011,9/24/2012,hours,Voluntarily Terminated,Production,Kissy Sullivan,20.0,LinkedIn,Fully Meets,3.02,3,0,5/15/2012,0,3
3,"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991,0,19,Production Technician I,MA,1886,09/27/88,F,Married,US Citizen,No,White,1/7/2008,NaN,N/A-StillEmployed,Active,Production,Elijiah Gray,16.0,Indeed,Fully Meets,4.84,5,0,1/3/2019,0,15
4,"Anderson, Carol",10069,0,2,0,5,5,3,0,50825,1,19,Production Technician I,MA,2169,09/08/89,F,Divorced,US Citizen,No,White,7/11/2011,9/6/2016,return to school,Voluntarily Terminated,Production,Webster Butler,39.0,Google Search,Fully Meets,5.00,4,0,2/1/2016,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,"Woodson, Jason",10135,0,0,1,1,5,3,0,65893,0,20,Production Technician II,MA,1810,05/11/85,M,Single,US Citizen,No,White,7/7/2014,NaN,N/A-StillEmployed,Active,Production,Kissy Sullivan,20.0,LinkedIn,Fully Meets,4.07,4,0,2/28/2019,0,13
307,"Ybarra, Catherine",10301,0,0,0,5,5,1,0,48513,1,19,Production Technician I,MA,2458,05/04/82,F,Single,US Citizen,No,Asian,9/2/2008,9/29/2015,Another position,Voluntarily Terminated,Production,Brannon Miller,12.0,Google Search,PIP,3.20,2,0,9/2/2015,5,4
308,"Zamora, Jennifer",10010,0,0,0,1,3,4,0,220450,0,6,CIO,MA,2067,08/30/79,F,Single,US Citizen,No,White,4/10/2010,NaN,N/A-StillEmployed,Active,IT/IS,Janet King,2.0,Employee Referral,Exceeds,4.60,5,6,2/21/2019,0,16
309,"Zhou, Julia",10043,0,0,0,1,3,3,0,89292,0,9,Data Analyst,MA,2148,02/24/79,F,Single,US Citizen,No,White,3/30/2015,NaN,N/A-StillEmployed,Active,IT/IS,Simon Roup,4.0,Employee Referral,Fully Meets,5.00,3,5,2/1/2019,0,11


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Employee_Name               311 non-null    object 
 1   EmpID                       311 non-null    int64  
 2   MarriedID                   311 non-null    int64  
 3   MaritalStatusID             311 non-null    int64  
 4   GenderID                    311 non-null    int64  
 5   EmpStatusID                 311 non-null    int64  
 6   DeptID                      311 non-null    int64  
 7   PerfScoreID                 311 non-null    int64  
 8   FromDiversityJobFairID      311 non-null    int64  
 9   Salary                      311 non-null    int64  
 10  Termd                       311 non-null    int64  
 11  PositionID                  311 non-null    int64  
 12  Position                    311 non-null    object 
 13  State                       311 non

### Preprocessing

In [4]:
df = data.copy()

In [5]:
df.isna().sum()

Employee_Name                   0
EmpID                           0
MarriedID                       0
MaritalStatusID                 0
GenderID                        0
EmpStatusID                     0
DeptID                          0
PerfScoreID                     0
FromDiversityJobFairID          0
Salary                          0
Termd                           0
PositionID                      0
Position                        0
State                           0
Zip                             0
DOB                             0
Sex                             0
MaritalDesc                     0
CitizenDesc                     0
HispanicLatino                  0
RaceDesc                        0
DateofHire                      0
DateofTermination             207
TermReason                      0
EmploymentStatus                0
Department                      0
ManagerName                     0
ManagerID                       8
RecruitmentSource               0
PerformanceSco

In [6]:
(df['GenderID'] == df['Sex'].apply(lambda x: 1 if x=='M ' else 0)).all()

True

In [7]:
# Drop unneeded columns
df = df.drop(['DateofTermination', 'TermReason', 'MaritalStatusID', 'Sex', 'PositionID', 'PerfScoreID', 'DeptID', 'EmpStatusID', 'ManagerID', 'EmploymentStatus','Employee_Name', 'EmpID'], axis=1)

In [8]:
df.isna().sum()

MarriedID                     0
GenderID                      0
FromDiversityJobFairID        0
Salary                        0
Termd                         0
Position                      0
State                         0
Zip                           0
DOB                           0
MaritalDesc                   0
CitizenDesc                   0
HispanicLatino                0
RaceDesc                      0
DateofHire                    0
Department                    0
ManagerName                   0
RecruitmentSource             0
PerformanceScore              0
EngagementSurvey              0
EmpSatisfaction               0
SpecialProjectsCount          0
LastPerformanceReview_Date    0
DaysLateLast30                0
Absences                      0
dtype: int64

In [9]:
{column: len(df[column].unique()) for column in df.columns}

{'MarriedID': 2,
 'GenderID': 2,
 'FromDiversityJobFairID': 2,
 'Salary': 308,
 'Termd': 2,
 'Position': 32,
 'State': 28,
 'Zip': 158,
 'DOB': 307,
 'MaritalDesc': 5,
 'CitizenDesc': 3,
 'HispanicLatino': 4,
 'RaceDesc': 6,
 'DateofHire': 101,
 'Department': 6,
 'ManagerName': 21,
 'RecruitmentSource': 9,
 'PerformanceScore': 4,
 'EngagementSurvey': 119,
 'EmpSatisfaction': 5,
 'SpecialProjectsCount': 9,
 'LastPerformanceReview_Date': 137,
 'DaysLateLast30': 7,
 'Absences': 20}

In [10]:
df.loc[:, [len(df[column].unique()) == 2 for column in df.columns]]

,MarriedID,GenderID,FromDiversityJobFairID,Termd
0,0,1,0,0
1,1,1,0,1
2,1,0,0,1
3,1,0,0,0
4,0,0,0,1
...,...,...,...,...
306,0,1,0,0
307,0,0,0,1
308,0,0,0,0
309,0,0,0,0


In [11]:
def encode_dates(df, columns_with_prefixes):
    df = df.copy()
    for column, prefix in columns_with_prefixes:
        df[column] = pd.to_datetime(df[column])

        df[prefix + '_year'] = df[column].apply(lambda x: x.year)
        df[prefix + '_month'] = df[column].apply(lambda x: x.month)
        df[prefix + '_day'] = df[column].apply(lambda x: x.day)

        df = df.drop(column, axis=1)
    return df

In [12]:
# Date encoding
date_columns = [
    ('DOB','DOB'),
    ('DateofHire', 'DOH'),
    ('LastPerformanceReview_Date', 'PRD')
]

df = encode_dates(df, date_columns)
df

/tmp/ipykernel_10231/2477862499.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column] = pd.to_datetime(df[column])


,MarriedID,GenderID,FromDiversityJobFairID,Salary,Termd,Position,State,Zip,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc,Department,ManagerName,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences,DOB_year,DOB_month,DOB_day,DOH_year,DOH_month,DOH_day,PRD_year,PRD_month,PRD_day
0,0,1,0,62506,0,Production Technician I,MA,1960,Single,US Citizen,No,White,Production,Michael Albert,LinkedIn,Exceeds,4.60,5,0,0,1,1983,7,10,2011,7,5,2019,1,17
1,1,1,0,104437,1,Sr. DBA,MA,2148,Married,US Citizen,No,White,IT/IS,Simon Roup,Indeed,Fully Meets,4.96,3,6,0,17,1975,5,5,2015,3,30,2016,2,24
2,1,0,0,64955,1,Production Technician II,MA,1810,Married,US Citizen,No,White,Production,Kissy Sullivan,LinkedIn,Fully Meets,3.02,3,0,0,3,1988,9,19,2011,7,5,2012,5,15
3,1,0,0,64991,0,Production Technician I,MA,1886,Married,US Citizen,No,White,Production,Elijiah Gray,Indeed,Fully Meets,4.84,5,0,0,15,1988,9,27,2008,1,7,2019,1,3
4,0,0,0,50825,1,Production Technician I,MA,2169,Divorced,US Citizen,No,White,Production,Webster Butler,Google Search,Fully Meets,5.00,4,0,0,2,1989,9,8,2011,7,11,2016,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,0,1,0,65893,0,Production Technician II,MA,1810,Single,US Citizen,No,White,Production,Kissy Sullivan,LinkedIn,Fully Meets,4.07,4,0,0,13,1985,5,11,2014,7,7,2019,2,28
307,0,0,0,48513,1,Production Technician I,MA,2458,Single,US Citizen,No,Asian,Production,Brannon Miller,Google Search,PIP,3.20,2,0,5,4,1982,5,4,2008,9,2,2015,9,2
308,0,0,0,220450,0,CIO,MA,2067,Single,US Citizen,No,White,IT/IS,Janet King,Employee Referral,Exceeds,4.60,5,6,0,16,1979,8,30,2010,4,10,2019,2,21
309,0,0,0,89292,0,Data Analyst,MA,2148,Single,US Citizen,No,White,IT/IS,Simon Roup,Employee Referral,Fully Meets,5.00,3,5,0,11,1979,2,24,2015,3,30,2019,2,1


In [13]:
df['PerformanceScore'].unique()

array(['Exceeds', 'Fully Meets', 'Needs Improvement', 'PIP'], dtype=object)

In [14]:
# Ordinal Encoding
def ordinal_encode(df, columns_with_orderings):
    df = df.copy()
    for column, ordering in columns_with_orderings:
        df[column] = df[column].apply(lambda x: ordering.index(x))
    return df

In [15]:
ordinal_columns = [
    ('PerformanceScore',['PIP', 'Needs Improvement', 'Fully Meets', 'Exceeds'])
]

df = ordinal_encode(df, ordinal_columns)

In [16]:
df

,MarriedID,GenderID,FromDiversityJobFairID,Salary,Termd,Position,State,Zip,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc,Department,ManagerName,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences,DOB_year,DOB_month,DOB_day,DOH_year,DOH_month,DOH_day,PRD_year,PRD_month,PRD_day
0,0,1,0,62506,0,Production Technician I,MA,1960,Single,US Citizen,No,White,Production,Michael Albert,LinkedIn,3,4.60,5,0,0,1,1983,7,10,2011,7,5,2019,1,17
1,1,1,0,104437,1,Sr. DBA,MA,2148,Married,US Citizen,No,White,IT/IS,Simon Roup,Indeed,2,4.96,3,6,0,17,1975,5,5,2015,3,30,2016,2,24
2,1,0,0,64955,1,Production Technician II,MA,1810,Married,US Citizen,No,White,Production,Kissy Sullivan,LinkedIn,2,3.02,3,0,0,3,1988,9,19,2011,7,5,2012,5,15
3,1,0,0,64991,0,Production Technician I,MA,1886,Married,US Citizen,No,White,Production,Elijiah Gray,Indeed,2,4.84,5,0,0,15,1988,9,27,2008,1,7,2019,1,3
4,0,0,0,50825,1,Production Technician I,MA,2169,Divorced,US Citizen,No,White,Production,Webster Butler,Google Search,2,5.00,4,0,0,2,1989,9,8,2011,7,11,2016,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,0,1,0,65893,0,Production Technician II,MA,1810,Single,US Citizen,No,White,Production,Kissy Sullivan,LinkedIn,2,4.07,4,0,0,13,1985,5,11,2014,7,7,2019,2,28
307,0,0,0,48513,1,Production Technician I,MA,2458,Single,US Citizen,No,Asian,Production,Brannon Miller,Google Search,0,3.20,2,0,5,4,1982,5,4,2008,9,2,2015,9,2
308,0,0,0,220450,0,CIO,MA,2067,Single,US Citizen,No,White,IT/IS,Janet King,Employee Referral,3,4.60,5,6,0,16,1979,8,30,2010,4,10,2019,2,21
309,0,0,0,89292,0,Data Analyst,MA,2148,Single,US Citizen,No,White,IT/IS,Simon Roup,Employee Referral,2,5.00,3,5,0,11,1979,2,24,2015,3,30,2019,2,1


In [17]:
def onehot_encode(df, columns_with_prefixes):
    df = df.copy()
    for column, prefix in columns_with_prefixes:
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [19]:
df.select_dtypes('object').columns

Index(['Position', 'State', 'MaritalDesc', 'CitizenDesc', 'HispanicLatino',
       'RaceDesc', 'Department', 'ManagerName', 'RecruitmentSource'],
      dtype='object')

In [20]:
nominal_columns = [
    ('Position', 'POS'),
    ('State', 'ST'),
    ('Zip', 'ZP'),
    ('MaritalDesc', 'MD'),
    ('CitizenDesc', 'CD'),
    ('HispanicLatino', 'HL'),
    ('RaceDesc', 'RD'),
    ('Department', 'DE'),
    ('ManagerName', 'MN'),
    ('RecruitmentSource', 'RS')
]

In [21]:
# One Hot Encoding
df = onehot_encode(df, nominal_columns)

In [22]:
df

,MarriedID,GenderID,FromDiversityJobFairID,Salary,Termd,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences,DOB_year,DOB_month,DOB_day,DOH_year,DOH_month,DOH_day,PRD_year,PRD_month,PRD_day,POS_Accountant I,POS_Administrative Assistant,POS_Area Sales Manager,POS_BI Developer,POS_BI Director,POS_CIO,POS_Data Analyst,POS_Data Analyst,POS_Data Architect,POS_Database Administrator,POS_Director of Operations,POS_Director of Sales,POS_Enterprise Architect,POS_IT Director,POS_IT Manager - DB,POS_IT Manager - Infra,POS_IT Manager - Support,POS_IT Support,POS_Network Engineer,POS_President & CEO,POS_Principal Data Architect,POS_Production Manager,POS_Production Technician I,POS_Production Technician II,POS_Sales Manager,POS_Senior BI Developer,POS_Shared Services Manager,POS_Software Engineer,POS_Software Engineering Manager,POS_Sr. Accountant,POS_Sr. DBA,POS_Sr. Network Engineer,ST_AL,ST_AZ,ST_CA,ST_CO,ST_CT,ST_FL,ST_GA,ST_ID,ST_IN,ST_KY,ST_MA,ST_ME,ST_MT,ST_NC,ST_ND,ST_NH,ST_NV,ST_NY,ST_OH,ST_OR,ST_PA,ST_RI,ST_TN,ST_TX,ST_UT,ST_VA,ST_VT,ST_WA,ZP_1013,ZP_1040,ZP_1420,ZP_1450,ZP_1460,ZP_1545,ZP_1550,ZP_1701,ZP_1721,ZP_1730,ZP_1731,ZP_1742,ZP_1749,ZP_1752,ZP_1760,ZP_1770,ZP_1773,ZP_1775,ZP_1776,ZP_1778,ZP_1801,ZP_1803,ZP_1810,ZP_1821,ZP_1824,ZP_1844,ZP_1845,ZP_1851,ZP_1864,ZP_1876,ZP_1880,ZP_1886,ZP_1887,ZP_1890,ZP_1901,ZP_1902,ZP_1905,ZP_1915,ZP_1938,ZP_1960,ZP_2021,ZP_2026,ZP_2027,ZP_2030,ZP_2031,ZP_2043,ZP_2044,ZP_2045,ZP_2048,ZP_2050,ZP_2056,ZP_2061,ZP_2062,ZP_2066,ZP_2067,ZP_2072,ZP_2081,ZP_2090,ZP_2093,ZP_2108,ZP_2109,ZP_2110,ZP_2119,ZP_2121,ZP_2122,ZP_2124,ZP_2125,ZP_2126,ZP_2127,ZP_2128,ZP_2129,ZP_2130,ZP_2132,ZP_2133,ZP_2134,ZP_2135,ZP_2136,ZP_2138,ZP_2140,ZP_2143,ZP_2145,ZP_2148,ZP_2149,ZP_2152,ZP_2154,ZP_2155,ZP_2169,ZP_2170,ZP_2171,ZP_2176,ZP_2180,ZP_2184,ZP_2189,ZP_2191,ZP_2199,ZP_2210,ZP_2301,ZP_2302,ZP_2324,ZP_2330,ZP_2343,ZP_2346,ZP_2351,ZP_2359,ZP_2360,ZP_2421,ZP_2445,ZP_2446,ZP_2451,ZP_2452,ZP_2453,ZP_2458,ZP_2459,ZP_2468,ZP_2472,ZP_2474,ZP_2478,ZP_2481,ZP_2492,ZP_2493,ZP_2703,ZP_2718,ZP_2747,ZP_2763,ZP_2908,ZP_3062,ZP_4063,ZP_5473,ZP_5664,ZP_6033,ZP_6040,ZP_6050,ZP_6070,ZP_6278,ZP_10171,ZP_19444,ZP_21851,ZP_27229,ZP_30428,ZP_33174,ZP_36006,ZP_37129,ZP_40220,ZP_43050,ZP_46204,ZP_58782,ZP_59102,ZP_78207,ZP_78230,ZP_78789,ZP_80820,ZP_83706,ZP_84111,ZP_85006,ZP_89139,ZP_90007,ZP_97756,ZP_98052,MD_Divorced,MD_Married,MD_Separated,MD_Single,MD_Widowed,CD_Eligible NonCitizen,CD_Non-Citizen,CD_US Citizen,HL_No,HL_Yes,HL_no,HL_yes,RD_American Indian or Alaska Native,RD_Asian,RD_Black or African American,RD_Hispanic,RD_Two or more races,RD_White,DE_Admin Offices,DE_Executive Office,DE_IT/IS,DE_Production,DE_Sales,DE_Software Engineering,MN_Alex Sweetwater,MN_Amy Dunn,MN_Board of Directors,MN_Brandon R. LeBlanc,MN_Brannon Miller,MN_Brian Champaigne,MN_David Stanley,MN_Debra Houlihan,MN_Elijiah Gray,MN_Eric Dougall,MN_Janet King,MN_Jennifer Zamora,MN_John Smith,MN_Kelley Spirea,MN_Ketsia Liebig,MN_Kissy Sullivan,MN_Lynn Daneault,MN_Michael Albert,MN_Peter Monroe,MN_Simon Roup,MN_Webster Butler,RS_CareerBuilder,RS_Diversity Job Fair,RS_Employee Referral,RS_Google Search,RS_Indeed,RS_LinkedIn,RS_On-line Web application,RS_Other,RS_Website
0,0,1,0,62506,0,3,4.60,5,0,0,1,1983,7,10,2011,7,5,2019,1,17,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

In [24]:
# Split df into X and y
y = df['Termd'].copy()
X = df.drop('Termd', axis=1).copy()

In [25]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=84)
X_train.shape, X_test.shape

((217, 291), (94, 291))

In [26]:
# Scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [27]:
X_train

,MarriedID,GenderID,FromDiversityJobFairID,Salary,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences,DOB_year,DOB_month,DOB_day,DOH_year,DOH_month,DOH_day,PRD_year,PRD_month,PRD_day,POS_Accountant I,POS_Administrative Assistant,POS_Area Sales Manager,POS_BI Developer,POS_BI Director,POS_CIO,POS_Data Analyst,POS_Data Analyst,POS_Data Architect,POS_Database Administrator,POS_Director of Operations,POS_Director of Sales,POS_Enterprise Architect,POS_IT Director,POS_IT Manager - DB,POS_IT Manager - Infra,POS_IT Manager - Support,POS_IT Support,POS_Network Engineer,POS_President & CEO,POS_Principal Data Architect,POS_Production Manager,POS_Production Technician I,POS_Production Technician II,POS_Sales Manager,POS_Senior BI Developer,POS_Shared Services Manager,POS_Software Engineer,POS_Software Engineering Manager,POS_Sr. Accountant,POS_Sr. DBA,POS_Sr. Network Engineer,ST_AL,ST_AZ,ST_CA,ST_CO,ST_CT,ST_FL,ST_GA,ST_ID,ST_IN,ST_KY,ST_MA,ST_ME,ST_MT,ST_NC,ST_ND,ST_NH,ST_NV,ST_NY,ST_OH,ST_OR,ST_PA,ST_RI,ST_TN,ST_TX,ST_UT,ST_VA,ST_VT,ST_WA,ZP_1013,ZP_1040,ZP_1420,ZP_1450,ZP_1460,ZP_1545,ZP_1550,ZP_1701,ZP_1721,ZP_1730,ZP_1731,ZP_1742,ZP_1749,ZP_1752,ZP_1760,ZP_1770,ZP_1773,ZP_1775,ZP_1776,ZP_1778,ZP_1801,ZP_1803,ZP_1810,ZP_1821,ZP_1824,ZP_1844,ZP_1845,ZP_1851,ZP_1864,ZP_1876,ZP_1880,ZP_1886,ZP_1887,ZP_1890,ZP_1901,ZP_1902,ZP_1905,ZP_1915,ZP_1938,ZP_1960,ZP_2021,ZP_2026,ZP_2027,ZP_2030,ZP_2031,ZP_2043,ZP_2044,ZP_2045,ZP_2048,ZP_2050,ZP_2056,ZP_2061,ZP_2062,ZP_2066,ZP_2067,ZP_2072,ZP_2081,ZP_2090,ZP_2093,ZP_2108,ZP_2109,ZP_2110,ZP_2119,ZP_2121,ZP_2122,ZP_2124,ZP_2125,ZP_2126,ZP_2127,ZP_2128,ZP_2129,ZP_2130,ZP_2132,ZP_2133,ZP_2134,ZP_2135,ZP_2136,ZP_2138,ZP_2140,ZP_2143,ZP_2145,ZP_2148,ZP_2149,ZP_2152,ZP_2154,ZP_2155,ZP_2169,ZP_2170,ZP_2171,ZP_2176,ZP_2180,ZP_2184,ZP_2189,ZP_2191,ZP_2199,ZP_2210,ZP_2301,ZP_2302,ZP_2324,ZP_2330,ZP_2343,ZP_2346,ZP_2351,ZP_2359,ZP_2360,ZP_2421,ZP_2445,ZP_2446,ZP_2451,ZP_2452,ZP_2453,ZP_2458,ZP_2459,ZP_2468,ZP_2472,ZP_2474,ZP_2478,ZP_2481,ZP_2492,ZP_2493,ZP_2703,ZP_2718,ZP_2747,ZP_2763,ZP_2908,ZP_3062,ZP_4063,ZP_5473,ZP_5664,ZP_6033,ZP_6040,ZP_6050,ZP_6070,ZP_6278,ZP_10171,ZP_19444,ZP_21851,ZP_27229,ZP_30428,ZP_33174,ZP_36006,ZP_37129,ZP_40220,ZP_43050,ZP_46204,ZP_58782,ZP_59102,ZP_78207,ZP_78230,ZP_78789,ZP_80820,ZP_83706,ZP_84111,ZP_85006,ZP_89139,ZP_90007,ZP_97756,ZP_98052,MD_Divorced,MD_Married,MD_Separated,MD_Single,MD_Widowed,CD_Eligible NonCitizen,CD_Non-Citizen,CD_US Citizen,HL_No,HL_Yes,HL_no,HL_yes,RD_American Indian or Alaska Native,RD_Asian,RD_Black or African American,RD_Hispanic,RD_Two or more races,RD_White,DE_Admin Offices,DE_Executive Office,DE_IT/IS,DE_Production,DE_Sales,DE_Software Engineering,MN_Alex Sweetwater,MN_Amy Dunn,MN_Board of Directors,MN_Brandon R. LeBlanc,MN_Brannon Miller,MN_Brian Champaigne,MN_David Stanley,MN_Debra Houlihan,MN_Elijiah Gray,MN_Eric Dougall,MN_Janet King,MN_Jennifer Zamora,MN_John Smith,MN_Kelley Spirea,MN_Ketsia Liebig,MN_Kissy Sullivan,MN_Lynn Daneault,MN_Michael Albert,MN_Peter Monroe,MN_Simon Roup,MN_Webster Butler,RS_CareerBuilder,RS_Diversity Job Fair,RS_Employee Referral,RS_Google Search,RS_Indeed,RS_LinkedIn,RS_On-line Web application,RS_Other,RS_Website
0,-0.849837,-0.907485,-0.309773,-0.208259,0.060081,0.363976,0.135254,-0.507088,-0.325094,1.349878,-0.585119,0.109117,-0.866386,0.158033,-0.462652,-1.346179,0.628807,-0.670422,0.188546,-0.068041,-0.118401,-0.318626,-0.118401,-0.068041,-0.068041,-0.118401,-0.068041,0.0,-0.137038,-0.068041,-0.068041,0.0,-0.068041,-0.068041,-0.068041,-0.068041,-0.153574,-0.137038,-0.068041,-0.068041,-0.208013,1.112256,-0.468082,-0.118401,-0.118401,0.0,-0.195646,-0.068041,0.0,-0.068041,-0.096449,-0.068041,-0.068041,-0.068041,-0.068041,-0.068041,-0.068041,-0.068041,-0.068041,0.0,0.0,0.344321,-0.068041,-0.068041,-0.068041,0.0,-0.068041,0.0,0.0,-0.068041,-0.068041,-0.068041,-0.068041,-0.068041,-0.096449,-0.068041,-0.068041,-0.068041,-0.068041,0.0,-0.068041,-0.096449,-0.096449,-0.096449,

### Training

In [28]:
models = {
    '   K-Nearest Neighbors': KNeighborsClassifier(),
    '   Logistic Regression': LogisticRegression(),
    'Support Vector Machine': SVC(),
    '         Decision Tree': DecisionTreeClassifier(),
    '        Neural Network': MLPClassifier()
}

In [29]:
for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

   K-Nearest Neighbors trained.
   Logistic Regression trained.
Support Vector Machine trained.
         Decision Tree trained.
        Neural Network trained.


### Results

In [30]:
for name, model in models.items():
    print(name + " Accuracy: {:.2f}%".format(model.score(X_test, y_test)*100))

   K-Nearest Neighbors Accuracy: 69.15%
   Logistic Regression Accuracy: 89.36%
Support Vector Machine Accuracy: 89.36%
         Decision Tree Accuracy: 100.00%
        Neural Network Accuracy: 74.47%
